##  Import Packages

In [ ]:
!spacy download en_core_web_lg
!spacy download en_core_web_sm

In [ ]:
from setfit import AbsaModel
from setfit import AbsaTrainer
from transformers import EarlyStoppingCallback
import datasets
from datasets import load_dataset
from setfit import TrainingArguments

## AbsaModel from setfit

In [ ]:
model = AbsaModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/all-mpnet-base-v2",
    spacy_model="en_core_web_sm",
)

In [ ]:
model = AbsaModel.from_pretrained("BAAI/bge-small-en-v1.5")

## Loading dataset

In [ ]:
# The training/eval dataset must have `text`, `span`, `label`, and `ordinal` columns
# Same dataset as ours but with span and ordinal

dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")
train_dataset = dataset.select(range(128))
eval_dataset = dataset.select(range(128, 256))

In [ ]:
train_dataset[1]

## Preparing trainer

In [ ]:
args = TrainingArguments(
    output_dir="models",
    num_epochs=5,
    use_amp=True,
    batch_size=128,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
)

In [ ]:
trainer = AbsaTrainer(
    model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

In [ ]:
trainer.train()

***** Running training *****
  Num unique pairs = 21624
  Batch size = 128
  Num epochs = 5
  Total optimization steps = 845


Step,Training Loss,Validation Loss,Embedding Loss,Rate
50,No log,No log,0.253100,0.000012
100,No log,No log,0.263500,0.000020
150,No log,No log,0.274700,0.000018
200,No log,No log,0.261200,0.000017
250,No log,No log,0.277900,0.000016
300,No log,No log,0.282200,0.000014


  0%|          | 0/205 [00:00<?, ?it/s]

  0%|          | 0/205 [00:00<?, ?it/s]

  0%|          | 0/205 [00:00<?, ?it/s]

...

  0%|          | 0/68 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

Loading best SentenceTransformer model from step 50.


## Evaluation

In [ ]:
metrics = trainer.evaluate(eval_dataset)
print(metrics)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
***** Running evaluation *****


***** Running evaluation *****


{'aspect': {'accuracy': 0.5973451327433629}, 'polarity': {'accuracy': 0.7322834645669292}}


## Prediction

In [ ]:
preds = model.predict(
    [
        "Best pizza outside of Italy and really tasty.",
        "The food variations are great and the prices are absolutely fair.",
        "Unfortunately, you have to expect some waiting time and get a note with a waiting number if it should be very full.",
    ]
)
print(preds)

[[{'span': 'pizza', 'polarity': 'positive'}, {'span': 'Italy', 'polarity': 'positive'}], [{'span': 'food variations', 'polarity': 'positive'}, {'span': 'prices', 'polarity': 'positive'}], [{'span': 'waiting time', 'polarity': 'neutral'}, {'span': 'note', 'polarity': 'neutral'}, {'span': 'waiting number', 'polarity': 'neutral'}]]
